<a href="https://colab.research.google.com/github/Nouf-Alabbasi/oKUmura_AI_Telecom_challenge/blob/main/Phi_2_Base_v_1_Telco_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### base RAG phi-2 code from [paper](https://www.semanticscholar.org/reader/aa32cce28aad1d04fea026860c3e2d4a218d9a57)
changes
- replaced the openAI model that generates answers with phi_2
- this model still uses OpenAi to generate the embeddings
- queries from our questions data set

## `Install requirements`

In [1]:
pip install faiss-cpu faiss-gpu ujson python-docx doc2docx openai gitpython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.7 MB/s eta 0:00:00


In [2]:
import os
import sys
import random
from transformers import AutoModelForCausalLM, AutoTokenizer
from doc2docx import convert
from docx import Document
import sqlite3
import faiss
import ujson
from copy import deepcopy
import traceback
import time
from openai import OpenAI
import numpy as np
import json
import git
from tqdm.auto import tqdm
import chardet
import ast
import openai
import re
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
import logging
import csv


## OpenAI **Key**

In [38]:
from google.colab import userdata
api_key = userdata.get('OpenAI')

Get documents and embeddings

In [4]:
folder_url = "https://huggingface.co/datasets/netop/Embeddings3GPP-R18"
clone_directory = "./3GPP-Release18"

if not os.path.exists(clone_directory):
    git.Repo.clone_from(folder_url, clone_directory)
    print("Folder cloned successfully!")
else:
    print("Folder already exists. Skipping cloning.")

Folder cloned successfully!


## Set the path to your RAG_telco_files folders

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:

path = '/content/drive/MyDrive/KU/RAG_telco_files'

##LLM(*Phi*-2)

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)

In [8]:
# model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
# tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")


In [9]:
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
def phi2(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)
    outputs = model.generate(**inputs, max_length=2048)
    # outputs = model.generate(**inputs, max_new_tokens=500, repetition_penalty=1.15)#, do_sample=True, top_p=0.90, num_return_sequences=3)
    text = tokenizer.batch_decode(outputs)[0]

    return text

prompt = '''def print_prime(n):
   """
   Print all primes between 1 and n
   """'''

phi2(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'def print_prime(n):\n   """\n   Print all primes between 1 and n\n   """\n   for i in range(2, n+1):\n       for j in range(2, i):\n           if i % j == 0:\n               break\n       else:\n           print(i)\n\nprint_prime(20)\n```\n\n## Exercises\n\n1. Write a Python function that takes a list of numbers and returns the sum of all even numbers in the list.\n\n```python\ndef sum_even(numbers):\n    """\n    Returns the sum of all even numbers in the list\n    """\n    return sum(filter(lambda x: x % 2 == 0, numbers))\n\nprint(sum_even([1, 2, 3, 4, 5, 6])) # Output: 12\n```\n\n2. Write a Python function that takes a list of strings and returns a new list containing only the strings that start with a vowel.\n\n```python\ndef filter_vowels(strings):\n    """\n    Returns a new list containing only the strings that start with a vowel\n    """\n    vowels = [\'a\', \'e\', \'i\', \'o\', \'u\']\n    return list(filter(lambda x: x[0].lower() in vowels, strings))\n\nprint(filter_vowels(

##Storage mechanism

In [11]:
class Storage:
    def __init__(self, db_name):
        try:
            self.db_name = db_name
            self.conn = sqlite3.connect(db_name, check_same_thread=False)  # This allows the connection to be used in multiple threads
            self.cursor = self.conn.cursor()
            self.optimize_database()
        except sqlite3.Error as e:
            print(f"Failed to connect to database {db_name}: {e}")
            raise e  # Re-raise exception after logging to handle it upstream if needed

    def __enter__(self):
        self.conn = sqlite3.connect(self.db_name, check_same_thread=False)
        self.cursor = self.conn.cursor()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()

    def optimize_database(self):
        self.cursor.execute("PRAGMA cache_size = -64000;")
        self.cursor.execute("PRAGMA journal_mode = WAL;")
        self.cursor.execute("PRAGMA synchronous = OFF;")

    def create_dataset(self, dataset_name):
        try:
            # Create a new table with 'id' as the primary key and 'data' for JSON storage, if it doesn't already exist.
            self.cursor.execute(f'''CREATE TABLE IF NOT EXISTS {dataset_name} (id TEXT PRIMARY KEY, data TEXT)''')
            self.conn.commit()  # Commit changes to the database.
        except sqlite3.Error as e:
            # Handle SQLite errors, e.g., syntax errors in the SQL command.
            print(dataset_name)
            print(f"Database error: {e}")
        except Exception as e:
            # Handle unexpected errors, keeping the program from crashing.
            print(f"Exception in create_dataset: {e}")


    def insert_dict(self, dataset_name, data_dict):
        # Extract 'id' from data dictionary.
        dict_id = data_dict.get('id')

        # Proceed only if 'id' exists and it's not already in the dataset.
        if dict_id is not None and not self.is_id_in_dataset(dataset_name, dict_id):
            try:
                # Convert dictionary to JSON string.
                data_json = ujson.dumps(data_dict)
                # Insert 'id' and JSON string into the specified dataset.
                self.cursor.execute(f"INSERT INTO {dataset_name} (id, data) VALUES (?, ?)", (dict_id, data_json,))
                self.conn.commit()  # Commit the transaction.
            except sqlite3.Error as e:
                # Handle SQLite errors during insert operation.
                print(f"Database error: {e}")
            except Exception as e:
                # Handle other unexpected errors.
                print(f"Exception in insert_dict: {e}")

    def insert_dict_new(self, dataset_name, document, index):
        import json
        # Insert a dictionary as a JSON string into the specified table
        try:
            data_str = json.dumps(document)
            # Check if the ID already exists in the table
            self.cursor.execute(f"SELECT id FROM {dataset_name} WHERE id = ?", (index,))
            existing_id = self.cursor.fetchone()
            if existing_id:
                print(f"ID {index} already exists in {dataset_name}. Clearing table and retrying insertion.")
                # Clear the table
                self.cursor.execute(f"DELETE FROM {dataset_name}")
                self.conn.commit()
                print(f"All existing data in {dataset_name} has been cleared.")
            # Insert the new data
            self.cursor.execute(f"INSERT INTO {dataset_name} (id, data) VALUES (?, ?)", (index, data_str))
            self.conn.commit()
            print(f"Document inserted successfully into {dataset_name} with ID {index}.")
        except Exception as e:
            print(f"Error inserting into table {dataset_name}: {e}")


    def insert_or_update_dict(self, dataset_name, data_dict):
        # Extract 'id' from data dictionary.
        dict_id = data_dict.get('id')

        # Proceed only if 'id' exists.
        if dict_id is not None:
            try:
                # Convert dictionary to JSON string.
                data_json = ujson.dumps(data_dict)
                # Use 'REPLACE INTO' to insert or update the row with the specified 'id'.
                self.cursor.execute(f"REPLACE INTO {dataset_name} (id, data) VALUES (?, ?)", (dict_id, data_json,))
                self.conn.commit()  # Commit the transaction.
            except sqlite3.Error as e:
                # Handle SQLite errors during insert/update operation.
                print(f"Database error: {e}")
            except Exception as e:
                # Handle other unexpected errors.
                print(f"Exception in insert_or_update_dict: {e}")
                print(traceback.format_exc())


    def is_id_in_dataset(self, dataset_name, dict_id):
        try:
            # Execute a SQL query to check if the given 'dict_id' exists in the 'dataset_name' table.
            self.cursor.execute(f"SELECT 1 FROM {dataset_name} WHERE id = ?", (dict_id,))
            # Return True if the ID exists, False otherwise.
            return self.cursor.fetchone() is not None
        except sqlite3.Error as e:
            # Handle SQLite errors, logging the issue and indicating the ID was not found.
            print(f"Database error: {e}")
            return False


    def store_faiss_data(self, identifier, index, data_mapping):
        try:
            # Serialize the FAISS index into bytes for storage.
            serialized_index = faiss.serialize_index(index).tobytes()
            # Convert the data mapping dictionary into a JSON string.
            json_data_mapping = ujson.dumps(data_mapping)

            # Ensure the storage table exists, creating it if necessary.
            self.cursor.execute('''CREATE TABLE IF NOT EXISTS faiss_index_data
                                (id TEXT PRIMARY KEY,
                                    faiss_index BLOB,
                                    data_mapping TEXT)''')

            # Insert the serialized index and JSON data mapping into the database.
            self.cursor.execute("INSERT INTO faiss_index_data (id, faiss_index, data_mapping) VALUES (?, ?, ?)",
                                (identifier, serialized_index, json_data_mapping,))
            self.conn.commit()  # Commit the changes to the database.
        except sqlite3.Error as e:
            # Log database-related errors.
            print(f"Database error: {e}")
        except Exception as e:
            # Log any other exceptions.
            print(f"Exception in store_faiss_data: {e}")


    def retrieve_faiss_data(self, identifier):
        try:
            # Correct the SQL query to fetch the faiss_index and data_mapping for the given identifier.
            # Remove the parentheses around the selected columns to ensure proper data retrieval.
            self.cursor.execute("SELECT faiss_index, data_mapping FROM faiss_index_data WHERE id = ?", (identifier,))
            row = self.cursor.fetchone()

            if row is not None:
                # Correctly deserialize the FAISS index from the binary data stored in the database.
                # Use faiss.deserialize_index directly on the binary data without converting it back to an array.
                index = faiss.deserialize_index(row[0])

                # Deserialize the JSON string back into a Python dictionary.
                data_mapping = ujson.loads(row[1])

                return index, data_mapping
            else:
                # Return None if no entry was found for the given identifier.
                return None, None
        except sqlite3.Error as e:
            # Log SQLite errors and return None to indicate failure.
            print(f"Database error: {e}")
            return None, None
        except Exception as e:
            # Log unexpected errors and return None to indicate failure.
            print(f"Exception in retrieve_faiss_data: {e}")

    def retrieve_dicts(self, dataset_name):
        # Properly quote the table name to prevent SQL injection and syntax errors.
        safe_dataset_name = f'"{dataset_name}"'
        start= time.time()
        # Execute a SQL query to fetch all records from the specified dataset table.
        self.cursor.execute(f"SELECT * FROM {safe_dataset_name}")
        rows = self.cursor.fetchall()
        end=time.time()
        print(f"-------------------------{end-start}")
        # Utilizing a faster JSON parsing library if significant JSON parsing overhead is detected
        start= time.time()
        a= [ujson.loads(row[1]) for row in rows]
        end=time.time()
        print(f"UJSON-------------------------{end-start}")

        return a

    def reset_database(self):
        try:
            # Retrieve the names of all tables in the database.
            self.cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
            tables = self.cursor.fetchall()

            # Iterate over each table name and drop the table to remove it from the database.
            for table in tables:
                self.cursor.execute(f"DROP TABLE {table[0]}")

            # Commit the changes to finalize the removal of all tables.
            self.conn.commit()
        except sqlite3.Error as e:
            # Handle and log any SQLite errors encountered during the operation.
            print(f"Database error: {e}")
        except Exception as e:
            # Handle and log any non-SQLite errors that may occur.
            print(f"Exception in reset_database: {e}")

    def get_dict_by_id(self, dataset_name, dict_id):
        try:
            # Execute SQL query to fetch the record with the specified ID from the given dataset.
            self.cursor.execute(f"SELECT data FROM {dataset_name} WHERE id = ?", (dict_id,))
            row = self.cursor.fetchone()

            # If the record exists, convert the JSON string in 'data' column back to a dictionary and return it.
            if row is not None:
                return ujson.loads(row[0])
            else:
                # Return None if no record was found with the given ID.
                return None
        except sqlite3.Error as e:
            # Log database-related errors and return None to indicate failure.
            print(f"Database error: {e}")
            Storage.get_dict_by_id(self, dataset_name, dict_id)

        except Exception as e:
            # Log any other exceptions that occur and return None to indicate failure.
            print(f"Exception in get_dict_by_id: {e}")

    def close(self):
        try:
            # Attempt to close the database connection.
            self.conn.close()
        except sqlite3.Error as e:
            # Log any SQLite errors encountered during the closing process.
            print(f"Database error: {e}")
        except Exception as e:
            # Log any other exceptions that might occur during closure.
            print(f"Exception in close: {e}")
            print(traceback.format_exc())



##Retrieve chunked documents

In [12]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def read_docx(file_path):
    """Read and extract text from a DOCX file."""
    try:
        doc = Document(file_path)
        return '\n'.join(para.text for para in doc.paragraphs)
    except Exception as e:
        logging.error(f"Failed to read DOCX file at {file_path}: {e}")
        return None

def get_documents(series_list, folder_path=r'/content/3GPP-Release18/Documents', storage_name='Documents.db', dataset_name="Standard"):
    """Retrieve and process documents from a folder, storing them in a database if not already present."""
    storage = Storage(f'./3GPP-Release18/{storage_name}')
    storage.create_dataset(dataset_name)

    document_ds = []
    file_list = []

    # Check and convert .doc files to .docx
    convert_docs_to_docx(folder_path)

    # Prepare list of .docx files for processing
    file_list = [f for f in tqdm(os.listdir(folder_path), desc="Filtering documents") if valid_file(f, series_list)]

    # Process each document
    for filename in tqdm(file_list, desc="Processing documents"):
        file_path = os.path.join(folder_path, filename)
        process_document(file_path, filename, storage, document_ds, dataset_name)

    storage.close()
    return document_ds

def convert_docs_to_docx(folder_path):
    """Convert .doc files in a folder to .docx format if any."""
    has_doc = any(f.endswith('.doc') for f in os.listdir(folder_path))
    if has_doc:
        convert(folder_path)

def valid_file(filename, series_list):
    """Check if a file should be processed based on its name and series list."""
    return filename.endswith(".docx") and not filename.startswith("~$") and (not filename[:2].isnumeric() or int(filename[:2]) in series_list)

def process_document(file_path, filename, storage, document_ds, dataset_name):
    """Process a single document file."""
    if storage.is_id_in_dataset(dataset_name, filename):
        data_dict = storage.get_dict_by_id(dataset_name, filename)
        document_ds.append(data_dict)
    else:
        content = read_docx(file_path)
        if content:
            data_dict = {'id': filename, 'text': content, 'source': filename}
            document_ds.append(data_dict)
            storage.insert_dict(dataset_name, data_dict)



## Chunking

In [41]:

def custom_text_splitter(text, chunk_size, chunk_overlap, word_split=False):
    #print("in custom_text_splitter func\n")
    """
    Splits a given text into chunks of a specified size with a defined overlap between them.

    This function divides the input text into chunks based on the specified chunk size and overlap.
    Optionally, it can split the text at word boundaries to avoid breaking words when 'word_split'
    is set to True. This is achieved by using a regular expression that identifies word separators.

    Args:
        text (str): The text to be split into chunks.
        chunk_size (int): The size of each chunk in characters.
        chunk_overlap (int): The number of characters of overlap between consecutive chunks.
        word_split (bool, optional): If True, ensures that chunks end at word boundaries. Defaults to False.

    Returns:
        list of str: A list containing the text chunks.
    """
    chunks = []
    start = 0
    separators_pattern = re.compile(r'[\s,.\-!?\[\]\(\){}":;<>]+')

    while start < len(text) - chunk_overlap:
        end = min(start + chunk_size, len(text))

        if word_split:
            match = separators_pattern.search(text, end)
            if match:
                end = match.end()

        if end == start:
            end = start + 1

        chunks.append(text[start:end])
        start = end - chunk_overlap

        if word_split:
            match = separators_pattern.search(text, start-1)
            if match:
                start = match.start() + 1

        if start < 0:
            start = 0
        # print(text[start:end])
    return chunks


def chunk_doc(doc):
    #print("in chunk_doc func\n")
    chunks= custom_text_splitter( doc["text"], 500, 25, word_split = True)
    return [{"text": chunk, "source": doc["source"]} for chunk in chunks]



##Embeddings

In [14]:
def get_embeddings(series_docs):
    #print("in get_embeddings func\n")
    """Add embeddings to each chunk of documents from pre-saved NumPy files."""
    for doc_key, doc_chunks in series_docs.items():
        try:
            # Load embeddings specific to each document series
            embeddings = np.load(f'./3GPP-Release18/Embeddings/Embeddings{doc_key}.npy')
        except FileNotFoundError:
            logging.error(f"Embedding file for {doc_key} not found.")
            continue
        except Exception as e:
            logging.error(f"Failed to load embeddings for {doc_key}: {e}")
            continue

        # Process each chunk within the document series
        updated_chunks = []
        for chunk in doc_chunks:
            for idx, single_chunk in enumerate(chunk):
                try:
                    single_chunk['embedding'] = embeddings[idx]
                    updated_chunks.append(single_chunk)
                except IndexError:
                    logging.warning(f"Embedding index {idx} out of range for {doc_key}.")
                except Exception as e:
                    logging.error(f"Error processing chunk {idx} for {doc_key}: {e}")

        series_docs[doc_key] = updated_chunks

    return series_docs



##Get Terms and abbreviations

In [15]:
def read_docx(file_path):
    """Reads a .docx file and categorizes its content into terms and abbreviations."""
    doc = Document(file_path)

    processing_terms = False
    processing_abbreviations = False
    start = 0
    terms_definitions = {}
    abbreviations_definitions = {}

    for para in doc.paragraphs:
        text = para.text.strip()
        if "References" in text:
            start += 1
        if start >=2:
            if "Terms and definitions" in text:
                processing_terms = True
                processing_abbreviations = False

            elif "Abbreviations" in text:
                processing_abbreviations = True
                processing_terms = False
            else:
                if processing_terms and ':' in text:
                    term, definition = text.split(':', 1)
                    terms_definitions[term.strip()] = definition.strip().rstrip('.')
                elif processing_abbreviations and '\t' in text:
                    abbreviation, definition = text.split('\t', 1)
                    if len(abbreviation)> 1:
                        abbreviations_definitions[abbreviation.strip()] = definition.strip()

    return terms_definitions, abbreviations_definitions

file_path = r"21905-h20.docx"
terms_definitions, abbreviations_definitions = read_docx(path+"/"+file_path)

def preprocess(text, lowercase=True):
    """Converts text and optionally converts to lowercase. Removes punctuation."""
    if lowercase:
        text = text.lower()
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for char in punctuations:
        text = text.replace(char, '')
    return text

def find_and_filter_terms(terms_dict, sentence):
    """Finds terms in the given sentence, case-insensitively, and filters out shorter overlapping terms."""
    lowercase_sentence = preprocess(sentence, lowercase=True)

    # Find all terms
    matched_terms = {term: terms_dict[term] for term in terms_dict if preprocess(term) in lowercase_sentence}

    # Filter out terms that are subsets of longer terms
    final_terms = {}
    for term in matched_terms:
        if not any(term in other and term != other for other in matched_terms):
            final_terms[term] = matched_terms[term]

    return final_terms

def find_and_filter_abbreviations(abbreviations_dict, sentence):
    """Finds abbreviations in the given sentence, case-sensitively, and filters out shorter overlapping abbreviations."""
    processed_sentence = preprocess(sentence, lowercase=False)
    words = processed_sentence.split()

    matched_abbreviations = {word: abbreviations_dict[word] for word in words if word in abbreviations_dict}

    final_abbreviations = {}
    sorted_abbrs = sorted(matched_abbreviations, key=len, reverse=True)
    for abbr in sorted_abbrs:
        if not any(abbr in other and abbr != other for other in sorted_abbrs):
            final_abbreviations[abbr] = matched_abbreviations[abbr]

    # print(final_abbreviations)
    return final_abbreviations

def find_terms_and_abbreviations_in_sentence(terms_dict, abbreviations_dict, sentence):
    """Finds, filters terms and abbreviations in the given sentence.
       Filters to prioritize longer terms and abbreviations."""
    matched_terms = find_and_filter_terms(terms_dict, sentence)
    matched_abbreviations = find_and_filter_abbreviations(abbreviations_dict, sentence)

    # Format matched terms and abbreviations for output
    formatted_terms = [f"{term}: {definition}" for term, definition in matched_terms.items()]
    formatted_abbreviations = [f"{abbr}: {definition}" for abbr, definition in matched_abbreviations.items()]

    return formatted_terms, formatted_abbreviations

def find_terms_and_abbreviations_in_sentence(terms_dict, abbreviations_dict, sentence):
    """Finds and filters terms or abbreviations in the given sentence.
       Abbreviations are matched case-sensitively, terms case-insensitively, and longer terms are prioritized."""
    processed_sentence = preprocess(sentence, lowercase=False)  # Preserve case for abbreviations
    matched_abbreviations = {abbr: abbreviations_dict[abbr] for abbr in abbreviations_dict if abbr in processed_sentence}

    # Find and filter terms
    matched_terms = find_and_filter_terms(terms_dict, sentence)

    # Format matched terms and abbreviations for output
    formatted_terms = [f"{term}: {definition}" for term, definition in matched_terms.items()]
    formatted_abbreviations = [f"{abbr}: {definition}" for abbr, definition in matched_abbreviations.items()]

    return formatted_terms, formatted_abbreviations

def get_def(sentence):
    formatted_terms, formatted_abbreviations = find_terms_and_abbreviations_in_sentence(terms_definitions, abbreviations_definitions, sentence)
    defined = []
    for term in formatted_terms:
        defined.append(term[:3])
    for abbreviation in formatted_abbreviations:
        defined.append(abbreviation[:3])


def define_TA_question(sentence):
    formatted_terms, formatted_abbreviations = find_terms_and_abbreviations_in_sentence(terms_definitions, abbreviations_definitions, sentence)
    terms= '\n'.join(formatted_terms)
    abbreviations= '\n'.join(formatted_abbreviations)
    question= f"""{sentence}\n
Terms and Definitions:\n
{terms}\n

Abbreviations:\n
{abbreviations}\n
"""
    return question

##Faiss Index

In [16]:

os.environ['OMP_NUM_THREADS'] = '8'

def create_faiss_index_IndexFlatIP(embeddings, data, source):
    #print("in create_faiss_index_IndexFlatIP func\n")
    """Create FAISS IndexFlatIP from embeddings and maps indices to data and source."""
    try:
        logging.info("Creating IndexFlatIP...")
        d = embeddings.shape[1]
        index = faiss.IndexFlatIP(d)
        index.add(embeddings)
        index_to_data_mapping = {i: data[i] for i in range(len(data))}
        index_to_source_mapping = {i: source[i] for i in range(len(source))}
        return index, index_to_data_mapping, index_to_source_mapping
    except Exception as e:
        logging.error(f"Error creating FAISS index: {e}")
        return None, None, None

def get_faiss_batch_index(embedded_batch):
    #print("in get_faiss_batch_index func\n")
    """Generate FAISS index from a batch of embeddings, handling missing embeddings by generating them."""
    try:
        source = [chunk['source'] for chunked_batch in embedded_batch for chunk in chunked_batch]
        embeddings = []
        data = []

        for doc in embedded_batch:
            embeddings_batch = []
            for chunk in doc:
                if 'embedding' in chunk:
                    embeddings_batch.append(chunk['embedding'])
                else:
                    embedding = generate_embedding_for_chunk(chunk)
                    if embedding is not None:
                        chunk['embedding'] = embedding
                        embeddings_batch.append(embedding)

            embeddings.extend(embeddings_batch)
            data.extend([chunk['text'] for chunk in doc])

        embeddings = np.array(embeddings, dtype=np.float32)
        return create_faiss_index_IndexFlatIP(embeddings, data, source)
    except Exception as e:
        logging.error(f"Failed to process batch for FAISS indexing: {e}")
        return None, None, None

def generate_embedding_for_chunk(chunk):
    #print("in generate_embedding_for_chunk func\n")
    """Generate embeddings for a chunk using the OpenAI API."""
    try:
        client = OpenAI()
        response = client.embeddings.create(
            input=chunk["text"],
            model="text-embedding-3-large"
        )
        return response['data'][0]['embedding']
    except Exception as e:
        logging.error(f"Failed to generate embedding for chunk: {chunk['text']}. Error: {e}")
        return None

##Retrieval

In [39]:
def search_faiss_index(faiss_index, query_embedding, k=5):
    #print("in search_faiss_index func\n")
    # Validate input parameters
    if not isinstance(query_embedding, np.ndarray) or query_embedding.ndim != 1:
        raise ValueError("query_embedding must be a 1D numpy array")
    if not isinstance(k, int) or k <= 0:
        raise ValueError("k must be a positive integer")

    # Reshape the query embedding for FAISS (FAISS expects a 2D array)
    query_embedding_reshaped = query_embedding.reshape(1, -1)

    # Perform the search
    D, I = faiss_index.search(query_embedding_reshaped, k)

    # Return the indices and distances of the nearest neighbors
    return I, D

def get_query_embedding_OpenAILarge(query_text, context=None):
    #print("in get_query_embedding_OpenAILarge func\n")
    try:
        if context is not None:
            query_text = f'{query_text}\n' + "\n".join(context)

        api_key = userdata.get('OpenAI')
        client = openai.OpenAI(api_key=api_key)
        response = client.embeddings.create(
            input=query_text,
            model="text-embedding-3-large",
            dimensions= 1024
            )
        query_embedding =  response.data[0].embedding
        return np.array(query_embedding, dtype=np.float32)
    except Exception as e:
        print(f"Error occurred in get_query_embedding_OpenAILarge: {e}")
        traceback.print_exc()  # Using `print_exc` for consistency

def find_nearest_neighbors_faiss(query_text, faiss_index, data_mapping, k, source_mapping,  context=None):
    #print("in find_nearest_neighbors_faiss func\n")
    try:
        query_embedding = get_query_embedding_OpenAILarge(query_text, context)

        I, D = search_faiss_index(faiss_index, query_embedding, k)

        nearest_neighbors = []
        for index in I[0]:
            if index < len(data_mapping):
                data = data_mapping.get(index, "Data not found")
                source = source_mapping.get(index, "Source not found")
                nearest_neighbors.append((index, data, source))
        return nearest_neighbors
    except Exception as e:
        print(f"Error in find_nearest_neighbors_faiss: {str(e)}")
        traceback.print_exc()
        return []


##Neural Network

In [18]:
class NNRouter(nn.Module):
    def __init__(self):
        super(NNRouter, self).__init__()
        self.layer1_1 = nn.Linear(1024, 768)
        self.layer1_2 = nn.Linear(768, 512)
        self.layer1_3 = nn.Linear(512, 256)
        self.dropout1 = nn.Dropout(0.2)

        self.layer2_1 = nn.Linear(18, 128)
        self.layer2_2 = nn.Linear(128, 256)
        self.dropout2 = nn.Dropout(0.05)

        self.batchnorm1 = nn.BatchNorm1d(256)
        self.batchnorm2 = nn.BatchNorm1d(256)

        self.alfa = nn.Parameter(torch.ones(1), requires_grad=True)
        self.beta = nn.Parameter(torch.ones(1), requires_grad=True)

        self.output_layer1 = nn.Linear(256, 128)
        self.output_layer2 = nn.Linear(128, 18)

        self.leaky_relu = nn.LeakyReLU(0.01)

    def forward(self, input_1, input_2):
        #print("in NNRouter.forward func\n")
        x1 = F.relu(self.layer1_1(input_1))
        x1 = self.dropout1(x1)
        x1 = F.relu(self.layer1_2(x1))
        x1 = self.dropout1(x1)
        x1 = F.relu(self.layer1_3(x1))
        x1 = self.batchnorm1(x1)

        x2 = F.relu(self.layer2_1(input_2))
        x2 = self.dropout2(x2)
        x2 = F.relu(self.layer2_2(x2))
        x2 = self.batchnorm2(x2)

        weighted_x1 = self.alfa * x1
        weighted_x2 = self.beta * x2

        combined = weighted_x1 + weighted_x2

        output = self.output_layer1(self.leaky_relu(combined))
        output = self.output_layer2(self.leaky_relu(output))

        return output

## Don't run - testing phi-2 for candidate asnwer genration

In [20]:
# # !pip install -q -U accelerate
# !pip install accelerate
!pip install git+https://github.com/huggingface/accelerate

  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-irdzvfio
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-irdzvfio
  Resolved https://github.com/huggingface/accelerate to commit c3f422699a5cb7665edd5420b8b106512c09a85d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-

In [21]:
from transformers import pipeline
model_name = "microsoft/phi-2"

pipe = pipeline(
    "text-generation",
    model=model_name,
    device_map="auto",
    trust_remote_code=True,
)
from IPython.display import Markdown


ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

In [ ]:

# prompt = "Please create a Python application that can change wallpapers automatically."
prompt = f"""
Provide all the possible answers to the fallowing question. Conisdering your knowledge and the text provided.
How does a supporting UE attach to the same core network operator from which it detached in a shared network? [3GPP Release 17]

Considering the fallowing context:

Retrieval 1:
...Network Operators (MNOs, NPN operators, fixed operators, etc.), Local service providers, individuals (users), owners of facilities or proprietors of business in which the local access will occur and third party service providers. These stakeholders will work together to provide local access to services.
The main activities that we will be concerned with in this study are first how to commission and decommission access, as including the relevant services. The access is not merely to a network but...
This retrieval is performed from the document 22844-i20.docx.


Retrieval 2:
...to Mobile Stations (MS) in idle mode".
[9]	ISBN 2-7461-1832-4: “UIC Project EIRENE System Requirements Specification”.
[10]	ETSI EN 301 515 V3.0.0 (2018-03): “Global System for Mobile communication (GSM); Requirements for GSM operation on railways”.
3	Definitions and abbreviations
3.1	Definitions
For the purposes of the present document, the terms and definitions given in 3GPP TR 21.905 [4] and the following apply. A term defined in the present document takes precedence over the definition of the...
This retrieval is performed from the document 22280-i50.docx.


Retrieval 3:
...F. Bush, and D. Bakken, (eds.), IEEE Vision for Smart Grid Communications: 2030 and Beyond, New York: IEEE, 2013.
[11]	US Department of Energy, "Communications Requirements of Smart Grid Technologies, 2010", accessed 14.08.20, http://energy.gov/sites/prod/files/gcprod/documents/Smart_Grid_Communications_Requirements_Report_10-05-2010.pdf
[12]	IT Process Wiki – The ITIL Wiki: Content is available according to Creative Commons Attribution-NonCommercial-ShareAlike 3.0 Germany License. Access 14.10.20.
[...
This retrieval is performed from the document 22867-i20.docx.


Retrieval 4:
...where they can get required network slice services, to minimize time period of service unavailability. E.g. in above service flow, if UE B2 is stuck at F1, the UE B2 is out of service until it moves to F2.
5.1.4	Post-conditions
Figure 5.1.4-1 shows on which frequency each UE camps on finally to get desirable network slice service.

Figure 5.1.4-1 UE status after-power on

5.1.5	Existing features partly or fully covering the use case functionality
When a UE is located in an area where there is an...
This retrieval is performed from the document 22835-i20.docx.


Retrieval 5:
...and a user roams into the shared network it shall be possible for that user to register with a core network operator (among the network sharing partners) that the user’s home operator has a roaming agreement with, even if the operator is not operating a radio access network in that area.
-	The selection of a core network operator among those connected to the shared radio access network can either be manual (i.e. performed by the user after obtaining a list of available core network operators) or...
This retrieval is performed from the document 22011-i50.docx.


Retrieval 6:
....
CPN	Customer Premises Network
eRG	Evolved Residential Gateway
PRAS	Premises Radio Access Station
4	Overview
Many operators around the world both have mobile network and fixed network operations. Operators provide triple-play, combining mobile communication, fixed telephony and broadband Internet, or even quad-play, adding TV, to customers in the consumer / residential market. In this market, operators can distinguish themselves by providing an optimal integration between the different services....
This retrieval is performed from the document 22858-i20.docx.


Retrieval 7:
...(this may be a 3GPP identity), UE capability of the UAV controller, position, owner identity, owner address, owner contact details, owner certification, UAV operator identity, UAV operator license, UAV operator certification, UAV pilot identity, UAV pilot license, UAV pilot certification and flight plan.
[R-5.1-005] The 3GPP system shall enable a UAS to send different UAS data to UTM based on the different authentication and authorizations level which are applied to the UAS.
NOTE 0: 	Subject to the...
This retrieval is performed from the document 22125-i00.docx.


Retrieval 8:
...Additional functionality may be required based on regulatory or operator policy requirements.
4.6	PWS-UE Requirements
4.6.1	General Requirements
PWS-UEs shall only be required to receive and present Warning Notifications in languages as presented by the Warning Notification Provider. Regional/regulatory requirements may require the Warning Notifications to be broadcast in multiple languages.
There shall be no requirement for language translation in the operator’s network or the UE.
It shall be possible...
This retrieval is performed from the document 22268-i30.docx.


Retrieval 9:
...flexible introduction of telecommunication services;
h)	to provide within the residential environment the capability to enable a pedestrian user to access all services normally provided by fixed networks;
i)	to provide within the office environment the capability to enable a pedestrian user to access all services normally provided by PBXs and LANs;
j)	to provide a substitute for fixed networks in areas of diverse population densities, under conditions approved by the appropriate national or regional...
This retrieval is performed from the document 22101-i50.docx.


Retrieval 10:
...the context of this TR, from a high-level 5G system point of view, the target scenarios include moving vehicles equipped with small on board base station (BS) relays providing 5G coverage and communication to UEs (inside the vehicle and/or in its vicinity), and connected wirelessly to the 5G network via RAN (donor) nodes.



Provide all the possible answers to the fallowing question. Conisdering your knowledge and the text provided.
Question How does a supporting UE attach to the same core network operator from which it detached in a shared network? [3GPP Release 17]\n


Please do not directly quote the provided context. Make sure none of the answers provided contradicts with your knowledge and have at most 50 characters each.

Output:
"""


print(phi2(prompt))
# outputs = pipe(
#     prompt,
#     max_new_tokens=300,
#     do_sample=True,
#     temperature=0.7,
#     top_k=50,
#     top_p=0.95,
# )
# Markdown(outputs[0]["generated_text"])

In [ ]:

# generate candidate answers using OpenAI
generated_output = client.chat.completions.create(
model = "gpt-3.5-turbo-1106",
messages = [
    {"role": "system", "content": "You are an expert at telecom knowledge. Be concise, precise and provide exact technical terms required."},
    {"role": "user", "content":  prompt},

],
)

In [ ]:

# prompt = "Please create a Python application that can change wallpapers automatically."
prompt = f"""
Provide all the possible answers to the fallowing question. Conisdering your knowledge and the text provided.
How does a supporting UE attach to the same core network operator from which it detached in a shared network? [3GPP Release 17]



Make sure none of the answers provided contradicts with your knowledge and have at most 100 characters each.

answer:
"""

outputs = pipe(
    prompt,
    max_new_tokens=300,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
)
Markdown(outputs[0]["generated_text"])

In [ ]:
print(outputs[0]["generated_text"])


In [ ]:
# for i in outputs:
#   print(i)
# import re

text =  outputs[0]["generated_text"]
parts = text.split("Answers:")
if len(parts) < 2:
    print("None")
else:
  # The second part contains the answers
  # print(parts[1])
  answers_text = parts[1].strip().lower()

  # Split the answers based on ". " to get individual answers
  answers = answers_text.split(".")

  # Clean up and filter out any empty answers
  answers = [answer.strip() for answer in answers if answer.strip()]

  print(type(answers))


##Query class

In [35]:
from transformers import AutoTokenizer, AutoModel
class Query:
    def __init__(self, query, context):
        self.id = id
        self.question = query
        self.query = query
        self.enhanced_query = query
        self.con_counter = {}
        self.topic_distr = []
        if isinstance(context, str):
            context = [context]
        self.context = context
        self.rowcontext = []
        self.context_source = []
        self.possible_sources = []
        self.wg = []
        self.source_hit={}
        self.document_accuracy = None


    def def_TA_question(self):
        self.query = define_TA_question(self.query)
        self.enhanced_query = self.query

    def candidate_answers(self):
            try:
                client = OpenAI()
                try:
                    row_context = f"""
                    Provide all the possible answers to the fallowing question. Conisdering your knowledge and the text provided.
                    Question {self.query}\n

                    Considering the following context:
                    {self.context}

                    Provide all the possible answers to the fallowing question. Conisdering your knowledge and the text provided.
                    Question {self.question}\n


                    Make sure none of the answers provided contradicts with your knowledge and have at most 100 characters each.
                    """
                    generated_output = client.chat.completions.create(
                    model = "gpt-3.5-turbo-1106",
                    messages = [
                        {"role": "system", "content": "You are an expert at telecom knowledge. Be concise, precise and provide exact technical terms required."},
                        {"role": "user", "content":  row_context},

                    ],
                    )
                    generated_output_str = generated_output.choices[0].message.content
                    # print(generated_output_str)
                    if generated_output_str != "NO":
                        self.context = generated_output_str
                        self.enhanced_query = self.query +'\n'+ self.context
                except Exception as e:
                    print(f"An error occurred: {e}")
            except:
                print("ERROR")
                print(traceback.format_exc())

    def get_embeddings_list(text_list):
        # Initialize the OpenAI client
        api_key = userdata.get('OpenAI')
        client = openai.OpenAI(api_key=api_key)
        # Request embeddings for the list of texts using a different, larger model
        response = client.embeddings.create(
                    input=text_list,
                    model="text-embedding-3-large",
                    dimensions=1024,
                )
        embeddings = []
        for i in range(len(response.data)):
            embeddings.append(response.data[i].embedding)

        text_embeddings= {}
        # print(len(text_list))
        # print(len(embeddings))
        for index in range(len(text_list)):
            text_embeddings[text_list[index]] = embeddings[index]
        return text_embeddings

    # def get_embeddings_list(text_list, model, tokenizer):
    #   embeddings = []
    #   # print(text_list)
    #   for text in text_list:
    #     input_ids = tokenizer.encode(text,
    #                                 return_tensors="pt",
    #                                 add_special_tokens=False,
    #                                 return_attention_mask=False,).to(device)
    #     # print(input_ids)
    #     with torch.no_grad():
    #         output = model(input_ids)
    #         logits = output.logits
    #         embedding = logits.mean(dim=1).detach().cpu().numpy()
    #         if embedding.shape[1] != 1024:
    #           embedding = np.resize(embedding, (1, 1024))  # Resize to 1024 dimensions
    #         print(embedding.shape)
    #         embeddings.append(embedding)
    #   text_embeddings= {}
    #   # print(len(text_list))
    #   # print(len(embeddings))
    #   for index in range(len(text_list)):
    #       text_embeddings[text_list[index]] = embeddings[index]
    #   return text_embeddings

    def inner_product(a, b):
        """Compute the inner product of two lists."""
        return sum(x * y for x, y in zip(a, b))

    def get_col2(embeddings_list):
        topics_with_series = [("Requirements (21 series): Focuses on the overarching requirements necessary for UMTS (Universal Mobile Telecommunications System) and later cellular standards, including GSM enhancements, security standards, and the general evolution of 3GPP systems. It covers vocabulary, security threats, UE capability requirements, and work items for various releases.", "21 series"),
            ("Service aspects ('stage 1') (22 series): This series details the initial specifications for services provided by the network, outlining the service requirements before the technical realization is detailed. It serves as the first step in defining what the network should provide.", "22 series"),
            ("Technical realization ('stage 2') (23 series): Focuses on the architectural and functional framework necessary to implement the services described in stage 1, providing a bridge to the detailed protocols and interfaces defined in stage 3​.,", "23 series"),
            ("Signalling protocols ('stage 3') - user equipment to network (24 series): Details the protocols and signaling procedures for communication between user equipment and the network, ensuring interoperability and successful service delivery.", "24 series"),
            ("Radio aspects (25 series): Covers the specifications related to radio transmission technologies, including frequency bands, modulation schemes, and antenna specifications, critical for ensuring efficient and effective wireless communication​.", "25 series"),
            ("CODECs (26 series): Contains specifications for voice, audio, and video codecs used in the network, defining how data is compressed and decompressed to enable efficient transmission over bandwidth-limited wireless networks.", "26 series"),
            ("Data (27 series): This series focuses on the data services and capabilities of the network, including specifications for data transmission rates, data service features, and support for various data applications.", "27 series"),
            ("Signalling protocols ('stage 3') - (RSS-CN) and OAM&P and Charging (overflow from 32.- range) (28 series): Addresses additional signaling protocols related to operation, administration, maintenance, provisioning, and charging, complementing the core signaling protocols outlined in the 24 series.", "28 series"),
            ("Signalling protocols ('stage 3') - intra-fixed-network (29 series): Specifies signaling protocols used within the fixed parts of the network, ensuring that various network elements can communicate effectively to provide seamless service to users.", "29 series"),
            ("Programme management (30 series): Relates to the management and coordination of 3GPP projects and work items, including documentation and specification management procedures​.", "30 series"),
            ("Subscriber Identity Module (SIM / USIM), IC Cards. Test specs. (31 series): Covers specifications for SIM and USIM cards, including physical characteristics, security features, and interaction with mobile devices, as well as testing specifications for these components​.", "31 series"),
            ("OAM&P and Charging (32 series): Focuses on operation, administration, maintenance, and provisioning aspects of the network, as well as the charging principles and mechanisms for billing and accounting of network services.", "32 series"),
            ("Security aspects (33 series): Details the security mechanisms and protocols necessary to protect network operations, user data, and communication privacy, including authentication, encryption, and integrity protection measures​.", "33 series"),
            ("UE and (U)SIM test specifications (34 series): Contains test specifications for User Equipment (UE) and (U)SIM cards, ensuring that devices and SIM cards meet 3GPP standards and perform correctly in the network​.", "34 series"),
            ("Security algorithms (35 series): Specifies the cryptographic algorithms used in the network for securing user data and signaling information, including encryption algorithms and key management procedures.", "35 series"),
            ("LTE (Evolved UTRA), LTE-Advanced, LTE-Advanced Pro radio technology (36 series): Details the technical specifications for LTE, LTE-Advanced, and LTE-Advanced Pro technologies, including radio access network (RAN) protocols, modulation schemes, and network architecture​.", "36 series"),
            ("Multiple radio access technology aspects (37 series): Addresses the integration and interoperability of multiple radio access technologies within the network, enabling seamless service across different types of network infrastructure.", "37 series"),
            ("Radio technology beyond LTE (38 series): Focuses on the development and specification of radio technologies that extend beyond the capabilities of LTE, aiming to improve speed, efficiency, and functionality for future cellular networks​.", "38 series")
        ]
        file_path = path + 'series_description.json'
        if os.path.isfile(file_path):
            # File exists, read the file and load the JSON content into series_dict
            with open(file_path, 'r') as file:
                series_dict = json.load(file)
        else:
            series_dict = {}
            for desc, series_index in topics_with_series:
                series_dict[series_index] = {}
                series_dict[series_index]["description"]= desc
                series_dict[series_index]["embeddings"]= Query.get_embeddings(desc)
            # File does not exist, write the series_dict to the file as JSON
            with open(file_path, 'w') as file:
                json.dump(series_dict, file, indent=4)

        similarity_coloumn = []
        for embeddings in embeddings_list:
            coef = []
            for series_id in series_dict:
                coef.append(Query.inner_product(embeddings, series_dict[series_id]['embeddings']))
            similarity_coloumn.append(coef)
        return similarity_coloumn

    def preprocessing_softmax(embeddings_list):
        embeddings = np.array(embeddings_list)
        similarity = np.array(Query.get_col2(embeddings))

        X_train_1_tensor = torch.tensor(embeddings, dtype=torch.float32)

        _similarity = torch.from_numpy(similarity)
        _similarity = torch.tensor(similarity, dtype=torch.float32)
        X_train_2_tensor= torch.nn.functional.softmax(10*_similarity, dim=-1)


        dataset = TensorDataset(X_train_1_tensor, X_train_2_tensor)
        gen = torch.Generator(device=device)
        dataloader = DataLoader(dataset, batch_size=128, shuffle=True, generator=gen)

        return dataloader

    def get_embeddings(text):
        # Initialize the OpenAI client
        client = OpenAI()
        api_key = userdata.get('OpenAI')
        client = openai.OpenAI(api_key=api_key)
        # Request embeddings for the list of texts using a different, larger model
        response = client.embeddings.create(
                    input=text,
                    model="text-embedding-3-large",
                    dimensions=1024,
                )
        return response.data[0].embedding


    def predict_wg(self, model_phi2, tokenizer):
        model = NNRouter()
        model.load_state_dict(torch.load(path+ r'/router_new.pth', map_location=device))
        model.to(device)
        model.eval()
        text_list = []
        # text_embeddings = Query.get_embeddings_list([self.enhanced_query], model_phi2, tokenizer)
        text_embeddings = Query.get_embeddings_list([self.enhanced_query])
        label_list = []
        embeddings = text_embeddings[self.enhanced_query]
        test_dataloader= Query.preprocessing_softmax([embeddings])
        with torch.no_grad():
            for X1, X2 in test_dataloader:
                # Move data to the same device as the model
                X1, X2 = X1.to(device), X2.to(device)
                original_labels_mapping = np.arange(21, 39)
                # X2 = X2.view(1024, 18)
                # print(X2.shape)
                outputs = model(X1, X2)
                top_values, top_indices = outputs.topk(5, dim=1)
                # Convert the indices to a numpy array
                predicted_indices = top_indices.cpu().numpy()
                predicted_labels = original_labels_mapping[predicted_indices]
                label_list=predicted_labels
        self.wg = label_list[0]
        # print(self.wg)

    def get_question_context_faiss(self, batch, k, model=model, tokenizer=tokenizer, use_context=False):
        # try:
        #     faiss_index, faiss_index_to_data_mapping, source_mapping = get_faiss_batch_index(batch)
        #     if use_context:
        #         result = find_nearest_neighbors_faiss(self.query, model, tokenizer, faiss_index, faiss_index_to_data_mapping, k, source_mapping= source_mapping, context=self.context)
        #     else: #                                   self.query, faiss_index, faiss_index_to_data_mapping, k, source_mapping, context=self.context)
        #         result = find_nearest_neighbors_faiss(self.query, model, tokenizer, faiss_index, faiss_index_to_data_mapping, k, source_mapping= source_mapping)
        try:
            faiss_index, faiss_index_to_data_mapping, source_mapping = get_faiss_batch_index(batch)
            if use_context:
                result = find_nearest_neighbors_faiss(self.query, faiss_index, faiss_index_to_data_mapping, k, source_mapping= source_mapping, context=self.context)
            else:
                result = find_nearest_neighbors_faiss(self.query, faiss_index, faiss_index_to_data_mapping, k, source_mapping= source_mapping)


            if isinstance(result, list):
                    self.context = []
                    self.context_source = []
                    for i in range(len(result)):
                        index, data, source = result[i]
                        self.context.append(f"\nRetrieval {i+1}:\n...{data}...\nThis retrieval is performed from the document {source}.\n")
                        self.context_source.append(f"Index: {index}, Source: {source}")
            else:
                self.context = result
        except Exception as e:
            print(f"An error occurred while getting question context: {e}")
            print(traceback.format_exc())
            self.context = "Error in processing"


# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Example usage
query_instance = Query("How does 5G impact telecommunications?", "5G is the fifth generation of cellular networks.")
query_instance.predict_wg(model, tokenizer)


Using device: cuda
1
1


##Generate

In [33]:
def generate(question):
        # Constructing the content context from the question object
        content = '\n'.join(question.context)
        prompt = f"""
Please answer to the following question:
{question.query}

Considering the following context:
{content}

Please answer to the following question:
{question.question}
        """

        logging.info("Generated system prompt for OpenAI completion.")

        # Extracting and cleaning the model's response
        predicted_answers_str = phi2(prompt)
        logging.info("Model response generated successfully.")

        context = f"The retrieved context provided to the LLM is:\n{content}"
        return predicted_answers_str, context, question.question



Check question

In [25]:
def find_first_number(string):
    """
    Finds the first sequence of digits in a given string and returns it as an integer.

    Parameters:
    - string: The string to search for numbers.

    Returns:
    - An integer representing the first sequence of digits found in the string.
    - Returns None if no digits are found.
    """
    try:
        # Using regular expression to search for the first occurrence of one or more digits
        match = re.search(r'\d+', string)
        if match:
            return int(match.group())  # Convert the found digits into an integer
        else:
            return None  # Return None if no digits are found
    except Exception as e:
        print(f"An error occurred while trying to find a number in the string: {e}")
        return None

def find_option_number(text):
    """
    Finds all the occurrences of numbers preceded by the word 'option' in a given text.

    Parameters:
    - text: The text to search for 'option' followed by numbers.

    Returns:
    - A list of strings, each representing a number found after 'option'. The numbers are returned as strings.
    - If no matches are found, an empty list is returned.
    """
    try:
        text =  text.lower()
        # Define a regular expression pattern to find 'option' followed by non-digit characters (\D*),
        # and then one or more digits (\d+)
        pattern = r'option\D*(\d+)'
        # Find all matches of the pattern in the text
        matches = re.findall(pattern, text)
        return matches  # Return the list of found numbers as strings
    except Exception as e:
        print(f"An error occurred while trying to find option numbers in the text: {e}")
        return []

def check_question_nonjson(question, answer, options):
    """
    This function checks if the answer provided for a non-JSON formatted question is correct.
    It dynamically selects the model based on the model_name provided and constructs a prompt
    for the AI model to generate an answer. It then compares the generated answer with the provided
    answer to determine correctness.

    Parameters:
    - question: A dictionary containing the question, options, and context.
    - model_name: Optional; specifies the model to use. Defaults to 'mistralai/Mixtral-8x7B-Instruct-v0.1'
    if not provided or if the default model is indicated.

    Returns:
    - A tuple containing the updated question dictionary and a boolean indicating correctness.
    """
    # Extracting options from the question dictionary.
    options_text = '\n'.join(options)

    content = '\n'.join(question.context)
    # Constructing the system prompt for the AI model.
    syst_prompt = f"""
    Instruct:
    Please provide the answers to the following multiple choice question.
    {question.query}

    Considering the following context:
    {content}

    Please provide the answers to the following multiple choice question.
    {question.question}
    The output should be in the format: Option <Option id>

    Options:
    Write only the option number corresponding to the correct answer:\n{options_text}

    Output:
    """

    # print(f"system prompt {syst_prompt}")
    predicted_answers_str = phi2(syst_prompt)
    predicted_answers_str = predicted_answers_str.split("Output:")[-1]
    # print(predicted_answers_str)
    print("Correct answer")
    print(answer)

    # Finding and comparing the predicted answer to the actual answer.
    answer_id = find_option_number(predicted_answers_str)
    print(f"tested option {answer_id}")

    if find_option_number(answer) == answer_id:
        print("Correct\n")
        return question, True, answer_id, syst_prompt
    else:
        print("Wrong\n")
        return question, False, answer_id, syst_prompt


##Telco-RAG

In [26]:
def TelcoRAG(query, answer, options,  model = model, tokenizer = tokenizer, api_key= 'sk-proj-bBeCQRP2o6QUpgt7UnwBT3BlbkFJPSiXDFCQQ79zdTrtULMi'):
    try:
        os.environ["OPENAI_API_KEY"] = 'sk-proj-bBeCQRP2o6QUpgt7UnwBT3BlbkFJPSiXDFCQQ79zdTrtULMi'
        os.environ["KMP_DUPLICATE_LIB_OK"] = 'TRUE'
        question = Query(query, [])

        question.def_TA_question()

        question.predict_wg(model, tokenizer)

        document_ds = get_documents(question.wg)

        print(len(document_ds))
        # Chunk documents based on provided chunk size and overlap
        Document_ds = [chunk_doc(doc) for doc in document_ds]

        series_doc = {'Summaries':[]}
        for series_number in question.wg:
            series_doc[f'Series{series_number}'] = []
            for doc in Document_ds:
                if doc[0]['source'][:2].isnumeric():
                    if int(doc[0]['source'][:2]) == series_number:
                        series_doc[f'Series{series_number}'].append(doc)
                else:
                    if doc not in series_doc['Summaries']:
                        series_doc['Summaries'].append(doc)

        series_docs = get_embeddings(series_doc)

        embedded_docs = []
        for serie in series_docs.values():
            embedded_docs.extend([serie])
        question.get_question_context_faiss(batch=embedded_docs, k=10, use_context=False)

        #question.candidate_answers()

        old_list =  question.wg
        question.predict_wg(model, tokenizer)
        new_series = {}
        for series_number in question.wg:
            if series_number not in old_list:
                new_series[f'Series{series_number}'] = []
                for doc in Document_ds:
                    if doc[0]['source'][:2].isnumeric():
                        if int(doc[0]['source'][:2]) == series_number:
                            new_series[f'Series{series_number}'].append(doc)
        new_series = get_embeddings(new_series)
        old_series={'Summaries': series_docs['Summaries']}
        for series_number in question.wg:
            if series_number in old_list:
                old_series[f'Series{series_number}'] = series_docs[f'Series{series_number}']
        embedded_docs = []
        for serie in new_series.values():
            embedded_docs.extend([serie])
        for serie in old_series.values():
            embedded_docs.extend([serie])
        question.get_question_context_faiss(batch=embedded_docs, k=10, use_context=True)

        answer_check = check_question_nonjson(question, answer , options)

                # pred answer , prompt         , correct_check
        return answer_check[2], answer_check[3], answer_check[1]

    except Exception as e:
        print(f"An error occurred: {e}")
        print(traceback.format_exc())


In [27]:
# response= TelcoRAG("can you list the slice requirements for a public safety slice in terms of quality of service, relaibility, latency and so on","low_latency", ["option 1: Reliability", "option 2: nothing"])

In [28]:
phi2("HEllo how are you?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'HEllo how are you?\', \'HEllo are you?\']\n    """\n    def capitalize_first_letter(s: str) -> str:\n        """\n        Given a string, returns the string with the first letter capitalized.\n        """\n        return s[0].upper() + s[1:]\n\n    return [capitalize_first_letter(s) + \'?\' if s.endswith(\'?\') else capitalize_first_letter(s) + \'!\' for s in li]\n\n<|endoftext|>'

## Run RAG phi-2 models on our data set

In [42]:

with open(path+"/TeleQnA.json", "r") as file:
  questions= json.load(file)



# split data
  # tbd


fields = ['Task','Question_ID','question', 'category', 'Answer_ID_RAG',"Prompt", 'actual answer',"match?"]
filename = path+"/_results_no_candidate_answers_openAI_embed.csv"


with open(filename, 'w', newline='') as results_file:
  writer = csv.writer(results_file)
  writer.writerow(fields)

  i = 0
  n = 50 # number of questions to test


  for ind, q in tqdm(questions.items()):
    # get all options
    options =[]
    options_dict = dict(filter(lambda item: "option" in item[0], q.items()))
    for opt in options_dict:

      # print(f"before {options}")
      # # if "None of the above" or "all of the above" replace with "not opt[1] and not opt[2]...."
      # last_opt = ""
      # if (options_dict[opt] == "None of the above"):
      #   # last_opt = "Not "
      #   last_opt += 'Not '+' and not '.join(options)
      #   options.append(last_opt)
      #   print(f"after {options}")

      # elif (options_dict[opt] == "All of the above"):
      #   last_opt += ' and '.join(options)
      #   options.append(last_opt)
      #   print(f"after {options}")

      # else:
        options.append(f"{opt}: {options_dict[opt]}")

    # response = response, prompt
    response= TelcoRAG(q["question"], q["answer"], options)

    # save to csv
    #  requireed format: Question_ID, Answer_ID, Task, actual answer, question
    #  current formate:  model, Question_ID, question text, RAG_Answer_ID, prompt, actual answer, match

    if (response is not None):
      row = ["phi_2", ind, q["question"],q["category"], response[0],response[1],q["answer"],response[2]]
    else:
      row = ["phi_2", ind, q["question"],q["category"], "None","None", q["answer"],"None"]

    writer.writerow(row)

    if (i>n):
      break;
    i+=1



  0%|          | 0/10000 [00:00<?, ?it/s]

1
1


Filtering documents:   0%|          | 0/553 [00:00<?, ?it/s]

Processing documents:   0%|          | 0/311 [00:00<?, ?it/s]

311
1
1


...

Filtering documents:   0%|          | 0/553 [00:00<?, ?it/s]

Processing documents:   0%|          | 0/318 [00:00<?, ?it/s]

318
1
1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



    Option 1

Terms and Definitions:

    Consider the following context:
    
    Retrieval 1:
   ...via an intermediary function (Service Communication Proxy - SCP),
    2)	NF/NF service set enabling the grouping of equivalent NF instances/NF service instances. The NF/NF Services within a NF/NF Service set can share the same context data thus improving the resiliency for processing any transaction. and 
    3)	
Correct answer
option 1: If a function is convex, its expectation is greater than or equal to the function applied to the expectation value
tested option ['1']
Correct



##Example Usage with TeleQnA

In [36]:
def choose_random_question(data):
    while True:
        random_question = random.choice(list(data.values()))
        # print(random_question)
        if '3GPP' in random_question['question']:
            return [random_question['question'], random_question['answer'],[random_question['option 1'],random_question['option 2'],random_question['option 3']]]
        else:
            continue

try:
    with open("TeleQnA.json", "r") as file:
        questions= json.load(file)
    question = choose_random_question(questions)
    print(question)
    response= TelcoRAG(question[0], question[1], question[2])
    print(response)
    print(f"""Generated response to the question:
          {response}
          # TelcoRAG doesn't return anything
Is:
            {response[0]} """)
except Exception as e:
    print("Encountered an error and moving to the next case.")
    print(traceback.format_exc())

Encountered an error and moving to the next case.
Traceback (most recent call last):
  File "<ipython-input-36-c7d6e7a402f2>", line 11, in <cell line: 10>
    with open("TeleQnA.json", "r") as file:
FileNotFoundError: [Errno 2] No such file or directory: 'TeleQnA.json'



In [ ]:
response.candidate_answers()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '\n                    Provide all the possible answers to the fallowing question. Conisdering your knowledge and the text provided.\n                    Question What is the purpose of the General requirements in the application layer architecture? [3GPP Release 18]\n\nTerms and Definitions:\n\nApplication: an application is a service enabler deployed by service providers, manufacturers or users. Individual applications will often be enablers for a wide range of services. (UMTS Forum report #2) [3]\n\n\nAbbreviations:\n\n3G: 3rd Generation\n3GPP: Third Generation Partnership Project\nGP: Guard Period\nPP: Point-to-Point\n\n\n\n\n                    Considering the fallowing context:\n                    1. Provisioning services. \n2. Defining service availability.\n3. Ensuring compatibility. \n4. Pre-arranging services for subscribers.\n\n                    Provide all the possible answers to the fallowing question. Conisdering your knowledge and the text provided